##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Treine a sua primeira rede neural: Classificação Básica

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/pt/r1/tutorials/keras/basic_classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Execute em Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/pt/r1/tutorials/keras/basic_classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Veja a fonte em GitHub</a>
  </td>
</table>

Note: A nossa comunidade TensorFlow traduziu estes documentos. Como as traduções da comunidade são *o melhor esforço*, não há garantias de que sejam uma reflexão exata e atualizada da [documentação oficial em Inglês](https://www.tensorflow.org/?hl=en). Se tem alguma sugestão para melhorar esta tradução, por favor envie um pull request para o repositório do GitHub [tensorflow/docs](https://github.com/tensorflow/docs). Para se voluntariar para escrever ou rever as traduções da comunidade, contacte a [lista docs@tensorflow.org](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs).

Este guia treina um modelo de uma rede neural para classificar imagens de roupa, como ténis ou camisolas. Não há problema se não perceber os detalhes, isto é uma revisão rápida de um programa completo em TensorFlow com os detalhes explicados à medida que avançamos.

Este guia usa [tf.keras](https://www.tensorflow.org/r1/guide/keras), uma API de alto nivel para construir e treinar modelos em TensorFlow.

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals, unicode_literals

# TensorFlow e tf.keras
import tensorflow as tf
from tensorflow import keras

# Bibliotecas de ajuda
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

## Importar o dataset Fashion MNIST

Este guia usa a base de dados [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) que contém 70.000 imagens a preto e branco de 10 categorias diferentes. As imagens apresentam peças de roupa individuais com pouca resolução (28 por 28 píxeis), como podem ser vistas aqui:

<table>
  <tr><td>
    <img src="https://tensorflow.org/images/fashion-mnist-sprite.png"
         alt="Fashion MNIST sprite"  width="600">
  </td></tr>
  <tr><td align="center">
    <b>Figure 1.</b> <a href="https://github.com/zalandoresearch/fashion-mnist">Fashion-MNIST samples</a> (by Zalando, MIT License).<br/>&nbsp;
  </td></tr>
</table>

Fashion MNIST tem a intenção de substituir a base de dados clássica [MNIST](http://yann.lecun.com/exdb/mnist/), que é usualmente utilizada como "Olá, Mundo" de programas de aprendizagem de máquina para visão computacional. A base de dados MNIST contém imagens de dígitos manuscritos (0, 1, 2, etc) num formato idêntico ao dos artigos de roupa que vamos utilizar aqui.

Este guia utiliza Fashion MNIST para variedade, e porque é um problema ligeiramente mais desafiante do que a MNIST. Ambas as bases de dados são relativamente pequenas e são utilizadas para verificar se os algoritmos trabalham como é esperado. São um bom início para testar e depurar o código.

Vamos utilizar 60.000 imagens para treinar a rede e 10.000 imagens para avaliar quão bem a rede aprendeu a classificar as imagens. Você pode acessar a base de dados Fashion MNIST diretamente a partir do TensorFlow, basta importar os dados:


In [0]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

Carregar a base de dados retorna quatro NumPy arrays:

* As `train_images` e `train_labels` arrays são o *training set*— os dados que o modelo utiliza para aprender
* O modelo é testado no *test set*, os arrays de `test_images` e os `test_labels`.

As images são 28x28 Numpy arrays, onde os valores de cada pixel variam entre 0 e 255. Os *labels* são um vetor de inteiros, que varia entre 0 e 9. Estes correspondem à *classe* de roupa que a imagem representa:


<table>
  <tr>
    <th>Label</th>
    <th>Class</th>
  </tr>
  <tr>
    <td>0</td>
    <td>T-shirt/top</td>
  </tr>
  <tr>
    <td>1</td>
    <td>Trouser</td>
  </tr>
    <tr>
    <td>2</td>
    <td>Pullover</td>
  </tr>
    <tr>
    <td>3</td>
    <td>Dress</td>
  </tr>
    <tr>
    <td>4</td>
    <td>Coat</td>
  </tr>
    <tr>
    <td>5</td>
    <td>Sandal</td>
  </tr>
    <tr>
    <td>6</td>
    <td>Shirt</td>
  </tr>
    <tr>
    <td>7</td>
    <td>Sneaker</td>
  </tr>
    <tr>
    <td>8</td>
    <td>Bag</td>
  </tr>
    <tr>
    <td>9</td>
    <td>Ankle boot</td>
  </tr>
</table>

Cada imagem é mapeada para um único label. Uma vez que os *nomes das classes* não estão incluídos na base de dados, guardamos aqui para mais tarde usar nos gráficos das images:

In [0]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

## Explorar os dados

Vamos explorar o formato da base de dados antes de treinar o modelo. De seguida, mostra-se que existem 60.000 imagens nos dados de treino, com cada imagem representada com 28x28 píxeis:


In [0]:
train_images.shape

De igual forma, existem 60.000 labels no dados de treino:

In [0]:
len(train_labels)

Cada label é um inteiro entre 0 e 9:

In [0]:
train_labels

Existem 10.000 imagens nos dados de teste. Cada imagem é novamente representada por 28 x 28 píxeis:

In [0]:
test_images.shape

E os dados de teste contém 10.000 labels de image:

In [0]:
len(test_labels)

## Pré-processamento dos dados

Os dados devem ser pré-processados antes de treinar a rede. Se observar a primeira imagem dos dados de treino, vai ver que o valor de cada píxel vai estar no intervalo entre 0 e 255:

In [0]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

Nós escalamos estes valores para estarem no intervalo de 0 a 1 antes de entrar no modelo neural. Para isso, dividimos os valores por 255. É importante que os dados de treino e os dados de teste sejam processados da mesma forma:

In [0]:
train_images = train_images / 255.0

test_images = test_images / 255.0

Mostre as primeiras 25 imagens dos *dados de treino* e a classe a que pertence por baixo de cada imagem. Verifique que os dados estão no formato correto e estamos prontos para contruir e treinar a rede.

In [0]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

## Construir o modelo

Construir a rede neural requer a configuração das camadas do modelo, e depois a compilação do modelo.

### Configurar as camadas

O bloco básico de uma rede neural é a *layer* (camada). As camadas extraem representações dos dados que as alimentam. E, esperançosamente, estas representações têm mais significado para o problema em mãos.

A maioria do deep learning consiste em encadear camadas simples. A maioria das camadas, como `tf.keras.layers.Dense`, têm parâmetros que são aprendidos durante o treino.

In [0]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

A primeira camada desta rede, `tf.keras.layers.Flatten`, transforma o formato das imagens de um vector 2D (de 28 por 28 píxeis), num vetor 1D de 28 * 28 = 784 píxeis. Considere esta camada como retirar as linhas de píxeis da imagem e alinhá-las. Esta camada não tem parâmetros para aprender; apenas re-formata os dados.

Depois de os píxeis estarem alinhados, a rede consiste numa sequência de duas camadas `tf.keras.layers.Dense`. Estas são categorizadas como *densely-connected*, ou *fully-connected*. A primeira camada `Dense` tem 128 nós (ou neurónios). A segunda (e última) é uma camada *softmax* com 10 nós - isto retorna um vetor com 10 valores de probabilidade que soma 1. Cada nó contém o valor que indica a probabilidade que a image atual pertence a uma das 10 classes.

### Compila o modelo

Antes de o modelo estar pronto para treinar, este precisa de mais algumas configurações. Estas são adicionadas no passo *compile*:

* *Função de custo - Isto mede quão bem o modelo classifica durante o treino. Nós queremos minimizar esta função para ajustar o modelo na direção correta*

* *Otimizador* - Isto é utilizado para atualizar o modelo com base nos dados e na função de custo.

* *Métricas* - São utilizadas para monitorizar os passos de treino e teste. O próximo exemplo usa *exatidão*, a fração das images que são corretamente classificadas.


In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## Treinar o modelo

Treinar o modelo da rede neural requer os seguintes passos:

1. Alimentar o modelo com os dados de treino, neste caso os vetores `train_images` e  `train_labels`.
2. O modelo aprende a associar as imagens e os labels.
3. Nós pedimos ao modelo para fazer uma previsão nos dados de teste - neste exemplo, o vetor `test_images`. Verificamos as previsões com o vetor de labels `test_labels`.

Para começar a treinar, chamamos o método `mode.fit`- o modelo vai ser a aproximado para descrever os dados de treino:

In [0]:
model.fit(train_images, train_labels, epochs=5)

À medida que o modelo treina, a função de custo e as métricas de exatidão são mostradas. Este modelo atinge uma exatidão de cerca de 0.88 (ou 88%) nos dados de treino.

## Avaliar a exatidão

De seguida, compare como o modelo se comporta nos dados de test:

In [0]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('Test accuracy:', test_acc)

Ao que parece, a exatidão nos dados de teste é um pouco menor do que a exatidão nos dados de treino. Esta lacuna entre a exatidão de treino e de teste é um exemplo de *overfitting*. Que significa quando um modelo de aprendizagem automática funciona pior em dados novos do que naqueles em que foi treinado.

## Fazer previsões

Com o modelo treinado, podemos utiliza-lo para fazer previsões sobre algumas imagens.

In [0]:
predictions = model.predict(test_images)

Aqui, o modelo previu o label para cada imagem nos dados de treino. Vamos observar a primeira previsão:

In [0]:
predictions[0]

A previsão é um vetor de 10 números. Este descrevem a "confiança" que o modelo tem de que esta imagem pertence a cada um dos 10 artigos diferentes de roupa. Podemos ver qual o rótulo apresenta a maior confiança:

In [0]:
np.argmax(predictions[0])

Então o modelo está mais confiante de que a image é *ankle boot*, ou `class_names[9]`. E podemos verificar o label do teste para verificar que está correto:

In [0]:
test_labels[0]

Podemos criar o gráfico para observar todos os 10 canais.

In [0]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

Vamos observar a imagem 0, as previsões, e o vector de previsão.

In [0]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions,  test_labels)
plt.show()

In [0]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions,  test_labels)
plt.show()

Vamos criar o gráfico com várias imagens e as suas previsões. As previsões correta encontram-se a azul e as incorretas a vermelho. O número indica a percentagem (em 100) para o label previsto. De notar, que pode estar errado mesmo quando o nível de confiança é elevado.

In [0]:
# Criar o gráfico das primeiras X imagens, o label previsto e o label correto
# Colorir as previsões corretas a azul e as incorretas a vermelho
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions, test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions, test_labels)
plt.show()

Finalmente, use o modelo treinado para fazer previsões sobre uma única imagem.

In [0]:
# Escolha uma imagem dos dados de teste
img = test_images[0]

print(img.shape)

Os modelos `tf.keras` estão otimizados para fazer previsões num *batch*, ou coleção, de exemplos de uma vez. Então, ainda que só estejamos a utilizar uma imagem temos de adicioná-la a uma lista:

In [0]:
# Adicione a imagem a batch onde é o único membro.
img = (np.expand_dims(img,0))

print(img.shape)

Agora faça uma previsão sobre a imagem:

In [0]:
predictions_single = model.predict(img)

print(predictions_single)

In [0]:
plot_value_array(0, predictions_single, test_labels)
_ = plt.xticks(range(10), class_names, rotation=45)

`model.predict` retorna uma lista de listas, uma para cada imagem do batch de dados. Selecione a previsão para a única image do batch:

In [0]:
np.argmax(predictions_single[0])

E tal como antes, o modelo prevê o label 9.